In [1]:
!git clone https://github.com/THUDM/GLM-4.git

Cloning into 'GLM-4'...
remote: Enumerating objects: 1665, done.
remote: Counting objects: 100% (496/496), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 1665 (delta 358), reused 269 (delta 267), pack-reused 1169 (from 2)
Receiving objects: 100% (1665/1665), 49.09 MiB | 20.33 MiB/s, done.
Resolving deltas: 100% (933/933), done.


In [2]:
# !pip install -r /content/GLM-4/basic_demo/requirements.txt

In [3]:
!cat /content/GLM-4/basic_demo/requirements.txt

cat: /content/GLM-4/basic_demo/requirements.txt: No such file or directory


In [4]:
!pip list|grep transformers

sentence-transformers                 4.1.0
transformers                          4.53.2


In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0' # 设置 GPU 编号，如果单机单卡指定一个，单机多卡指定多个 GPU 编号
MODEL_PATH = "THUDM/glm-4-9b-chat-hf"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)


model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="auto"
).eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/20.0M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [6]:
model

GlmForCausalLM(
  (model): GlmModel(
    (embed_tokens): Embedding(151552, 4096, padding_idx=151329)
    (layers): ModuleList(
      (0-39): 40 x GlmDecoderLayer(
        (self_attn): GlmAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (k_proj): Linear(in_features=4096, out_features=256, bias=True)
          (v_proj): Linear(in_features=4096, out_features=256, bias=True)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GlmMLP(
          (gate_up_proj): Linear(in_features=4096, out_features=27392, bias=False)
          (down_proj): Linear(in_features=13696, out_features=4096, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): GlmRMSNorm((4096,), eps=1.5625e-07)
        (post_attention_layernorm): GlmRMSNorm((4096,), eps=1.5625e-07)
      )
    )
    (norm): GlmRMSNorm((4096,), eps=1.5625e-07)
    (rotary_emb): GlmRotaryEmbedding()
  )
  (lm_head): Linear(in_

In [7]:
query = "请问5+15等于多少"

inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )

inputs = inputs.to(device)
#do_sample是否进行随机采样的作用
gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


5 + 15 等于 20。


In [ ]:
query = "你可以做什么，你的最长上下文是多少"

inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )
print(inputs)
inputs = inputs.to(device)
gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

{'input_ids': tensor([[151331, 151333, 151336,    198, 104114, 105520,   3837,  99444, 115085,
         102090,  98400, 117956, 151337]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


与GLM3的差异

从您提供的结构来看，`GlmForCausalLM` 和 `ChatGLMForConditionalGeneration` 都是基于 GLM（General Language Model）架构的模型，但二者在层数、注意力机制、规范化和一些关键的模块设计上有一些差异。这些变化反映了两种模型在不同任务上做出的结构性优化和调整。以下是详细的对比与分析：

### 1. **词嵌入层 (Embedding Layer)**
   - **`GlmForCausalLM`**：`embed_tokens` 的词汇表大小为 151552，嵌入维度为 4096，且设定了 `padding_idx` 为 151329。
   - **`ChatGLMForConditionalGeneration`**：`word_embeddings` 的词汇表大小为 65024，嵌入维度同样为 4096。
   
   **差异**：`GlmForCausalLM` 的词汇表大小明显更大（151552 vs 65024），可能是为了覆盖更广泛的词汇或扩展到更多的任务领域。此外，`GlmForCausalLM` 的 `padding_idx` 指定了一个特殊的填充索引，使得模型在处理不同长度的输入时能够更好地忽略填充位置。

### 2. **层数 (Layers)**
   - **`GlmForCausalLM`**：包含 40 层 `GlmDecoderLayer`。
   - **`ChatGLMForConditionalGeneration`**：包含 28 层 `GLMBlock`。
   
   **差异**：`GlmForCausalLM` 的层数更多（40 vs 28），这表明 `GlmForCausalLM` 更适合处理复杂任务，能够捕获更深层次的特征表示。此外，层数的增加可能会带来更好的语言建模效果，但同时也增加了计算复杂度。

### 3. **自注意力机制 (Self-Attention Mechanism)**
   - **`GlmForCausalLM`**：
     - 使用 `GlmSdpaAttention`，其中 `q_proj` 和 `o_proj` 的维度为 4096，而 `k_proj` 和 `v_proj` 的维度为 256。
   - **`ChatGLMForConditionalGeneration`**：
     - 使用 `SelfAttention`，其中 `query_key_value` 是一个维度为 4608 的线性层，显然这是将 `q`、`k`、`v` 的计算组合在了一起。
   
   **差异**：`GlmForCausalLM` 将 `Q`、`K`、`V` 的维度区分开来，`Q` 和 `O` 维度为 4096，而 `K` 和 `V` 仅为 256，这种设计更节省内存和计算成本，可能是为了支持更深的层次和更大的模型规模。相比之下，`ChatGLMForConditionalGeneration` 将 `Q`、`K`、`V` 合并计算，且所有特征的维度保持一致（4096），这在实现上较为简洁。

### 4. **多层感知机 (MLP)**
   - **`GlmForCausalLM`**：
     - 使用 `GlmMLP`，包含 `gate_up_proj` 和 `down_proj` 两个线性层，且激活函数为 `SiLU`。输入维度为 4096，扩展维度达到 27392。
   - **`ChatGLMForConditionalGeneration`**：
     - 使用 `MLP`，结构上包含 `dense_h_to_4h` 和 `dense_4h_to_h` 两个线性层，输入和输出维度相同，扩展维度也为 27392。
   
   **差异**：二者的 MLP 结构基本一致，主要区别在于激活函数的选择。`GlmForCausalLM` 采用了 `SiLU` 激活函数，而 `ChatGLMForConditionalGeneration` 没有提到特定的激活函数，可能默认为 ReLU 或其他激活函数。`SiLU`（Sigmoid Linear Unit）通常在深层网络中表现较好，能够更好地学习复杂的非线性关系。

### 5. **规范化层 (Normalization Layers)**
   - **`GlmForCausalLM`**：采用 `GlmRMSNorm`，并且 `eps` 的设置非常小（1.5625e-07）。
   - **`ChatGLMForConditionalGeneration`**：采用常规的 `RMSNorm`，并未指定 `eps` 值。
   
   **差异**：二者都使用 `RMSNorm` 作为规范化层，但 `GlmForCausalLM` 的 `eps` 更小，可能是为了增加模型的数值稳定性。

### 6. **位置编码 (Positional Encoding)**
   - **`GlmForCausalLM`**：使用了 `GlmRotaryEmbedding`，是一种旋转位置编码，用于增强长序列的建模能力。
   - **`ChatGLMForConditionalGeneration`**：使用了 `RotaryEmbedding`，也是旋转位置编码，但未显示出专门为 `GLM` 优化的版本。
   
   **差异**：两者都使用了旋转位置编码，但 `GlmForCausalLM` 使用了特别优化的 `GlmRotaryEmbedding`，可能包含更适合因果语言模型的增强或优化。

### 7. **输出层 (Output Layer)**
   - **`GlmForCausalLM`**：`lm_head` 是一个输出维度为 151552 的线性层，匹配词汇表大小。
   - **`ChatGLMForConditionalGeneration`**：`output_layer` 的输出维度为 65024，对应较小的词汇表。
   
   **差异**：由于 `GlmForCausalLM` 具有更大的词汇表，因此其输出层维度也更大。这使得该模型能够处理更多样的词汇或符号，适应更广泛的应用场景。

### 总结
- **规模和深度**：`GlmForCausalLM` 具有更多层数和更大的词汇表，适合更复杂的任务和更广泛的语言应用。
- **注意力机制**：`GlmForCausalLM` 将 `Q`、`K`、`V` 的维度分开，优化了计算和内存开销，而 `ChatGLMForConditionalGeneration` 则使用统一维度。
- **激活函数**：`GlmForCausalLM` 使用 `SiLU` 激活，可能在更深的层次中更稳定。
- **规范化层**：`GlmForCausalLM` 采用了更小的 `eps`，提高数值稳定性。
- **位置编码**：二者均使用旋转位置编码，但 `GlmForCausalLM` 中的版本可能更优化。

总体而言，`GlmForCausalLM` 在结构设计上更偏向复杂任务处理，并对资源使用做了优化，而 `ChatGLMForConditionalGeneration` 则是相对轻量的生成模型，适合特定应用场景。